Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch # é a base
import torch.nn as nn # importa as redes neurais
import torch.optim as optim #otimiza as redes

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()
#normaliza por transformação linear

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)
#retorna 0 ou 1

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)
# usar o float 32, em especifico o 32, pq ele funciona bem e ponto

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):    
    def __init__(#arquitetura da rede
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__() 

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1] 
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
# o shape vai descrever o número de dimensões e seus termos internos
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[-0.2289, -0.2837, -0.0638,  0.3429, -0.1030,  0.2184],
        [-0.0369, -0.2156, -0.1036,  0.3125,  0.0212, -0.3232],
        [ 0.3136,  0.3814,  0.2383,  0.2152,  0.0065,  0.3110],
        [-0.0675, -0.0080, -0.2146,  0.0765, -0.3115,  0.1256],
        [-0.1146,  0.3244,  0.2797, -0.0654,  0.0447, -0.2896],
        [ 0.3985, -0.1851, -0.0152,  0.1114, -0.2850,  0.0848],
        [ 0.1513, -0.3686, -0.2511, -0.3905,  0.2249, -0.0247],
        [ 0.2715,  0.0597, -0.2843,  0.0815,  0.1572, -0.3084],
        [ 0.1582, -0.3115,  0.1198, -0.2434, -0.2833,  0.0925],
        [ 0.0558, -0.3599,  0.0490,  0.1649,  0.1063,  0.3012],
        [-0.0866,  0.1017, -0.0858,  0.3335, -0.2005, -0.1484],
        [-0.0766,  0.0942, -0.2864, -0.3338,  0.0334, -0.1520],
        [-0.0087, -0.3898, -0.0840,  0.1443, -0.1475, -0.4062],
        [-0.2392, -0.1942,  0.0719,  0.0541, -0.1383,  0.1484],
        [ 0.2169, -0.0354,  0.3755, -0.3155,  0.1287, -0.0657],
        [ 0.1747, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[-0.1867],
        [-0.1827],
        [-0.1787],
        ...,
        [-0.1835],
        [-0.1813],
        [-0.1840]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente automático
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.1887)
1 tensor(0.1748)
2 tensor(0.1617)
3 tensor(0.1495)
4 tensor(0.1382)
5 tensor(0.1279)
6 tensor(0.1186)
7 tensor(0.1102)
8 tensor(0.1026)
9 tensor(0.0955)
10 tensor(0.0890)
11 tensor(0.0832)
12 tensor(0.0780)
13 tensor(0.0733)
14 tensor(0.0692)
15 tensor(0.0656)
16 tensor(0.0625)
17 tensor(0.0599)
18 tensor(0.0575)
19 tensor(0.0554)
20 tensor(0.0536)
21 tensor(0.0519)
22 tensor(0.0505)
23 tensor(0.0492)
24 tensor(0.0481)
25 tensor(0.0471)
26 tensor(0.0463)
27 tensor(0.0456)
28 tensor(0.0450)
29 tensor(0.0445)
30 tensor(0.0440)
31 tensor(0.0437)
32 tensor(0.0434)
33 tensor(0.0432)
34 tensor(0.0430)
35 tensor(0.0429)
36 tensor(0.0428)
37 tensor(0.0427)
38 tensor(0.0426)
39 tensor(0.0425)
40 tensor(0.0423)
41 tensor(0.0422)
42 tensor(0.0421)
43 tensor(0.0419)
44 tensor(0.0417)
45 tensor(0.0415)
46 tensor(0.0413)
47 tensor(0.0411)
48 tensor(0.0409)
49 tensor(0.0406)
50 tensor(0.0404)
51 tensor(0.0401)
52 tensor(0.0399)
53 tensor(0.0397)
54 tensor(0.0394)
55 tensor(0.0392)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1513.64335537]
[1409.99999118] [1564.07674599]
[1128.99997044] [1263.98701978]
[12991.99967575] [11723.31410599]
[14720.0001564] [14590.79441071]
[2938.99994564] [2698.54649734]
[6635.00014687] [8041.58315849]
[11215.99973297] [9363.1509304]
[1040.99999189] [1511.25380039]
[5040.00003624] [5169.89481163]
[9541.99988747] [8603.70736313]
[3792.99988461] [5307.28292274]
[1002.0000155] [769.00724292]
[2542.00005341] [1651.05544472]
[6788.00001717] [6643.62404251]
[9386.00025749] [8553.91201401]
[1245.99996853] [1727.77642059]
[997.00000262] [582.87827373]
[708.00000262] [774.02985597]
[1435.99997544] [1512.50397587]
[2590.00006676] [2539.5123806]
[1392.99997497] [1748.43601131]
[11755.99946976] [9405.97881126]
[2007.99997377] [1448.15101409]
[2707.99998474] [2584.2904911]
[631.00000417] [442.60953617]
[652.99999881] [988.91782212]
[2492.00006247] [2581.79344749]
[594.99999416] [354.11821055]
[760.0000056] [1216.37809968]
[1179.00003028] [1194.06621575]
[1789.9999361] [1844

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [904.27328038]
[628.00000334] [610.44001269]
[442.00000328] [686.98101401]
[2284.99997044] [2566.29579163]
[1040.99999189] [952.73466802]
[809.99999654] [1310.05085969]
[3049.99987316] [3415.78864956]
[720.00000596] [626.8481524]
[877.99999249] [481.79320788]
[1292.00000429] [1695.09105682]
[7056.00004578] [4961.91919136]
[4176.99999142] [5903.43196487]
[13701.00020599] [15157.52629471]
[2948.0000515] [4053.04823494]
[16732.99985504] [14102.37598801]
[1815.00000048] [2200.51197624]
[16220.00016022] [12049.1424675]
[5967.00019073] [4706.94871426]
[473.00000042] [518.59180331]
[1078.9999795] [1132.40522408]
[452.00000319] [633.05493188]
[801.99998283] [811.32551742]
[2458.99994993] [2433.60465813]
[2492.99998236] [2647.04455948]
[582.00000203] [672.10968614]
[1405.9999671] [1482.21677995]
[8715.99991035] [11450.08572006]
[7634.00004578] [6584.86248779]
[506.99999839] [599.47906613]
[665.99999094] [774.79330087]
[1815.00000048] [1670.69031048]
[707.00001383] [585.91861725]


E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1386.6495390783712


## Conclusão



Nesse notebook fizemos uma rede neural MLP otimizada que é mais complexa e versátil, com o pytorch, que utiliza o método Adam de otimização para treinar, o que ajuda a rede a fazer a melhor previsão com o passar das épocas.

Em alguns tópicos de discussões que acho importante;
- Quando precisarmos utilizarmos redes neurais na vida, não será necessário fazer uma do 0, afinal existem muitas redes prontas na internet, apenas vamos aplicá-las aos nossos dados.
- Método de otimização melhoram o treino das redes.
- As redes gostam de dados que variam entre 0 e 1 ou -1 e 1.
- O loss diminui conforme os treinos são feitos, então temos que ter cuidado com overfitting de dados, o que nos leva a ter que buscar um número ideal de épocas com outros métodos com dados de validação, ou seja, pegaremos a quantidade de épocas através do ponto mínimo dos dados de validação.
- Quanto mais você treina a rede (milhares de épocas) melhor a previsão ficará.

## Playground

